In [29]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [39]:
df=pd.read_excel('Data-reduced.xlsx')

In [40]:
#df = df.sample(500)
df = df[(df['Check FC'] == 1) | (df['Check FC'].isna())]
df=df[['DESCRIPTION','Herstellung']]

df.to_csv('data.csv', index=False)
df.head()

,DESCRIPTION,Herstellung
0,.Smoothfood Brot 48x70g,"Brot, übrige"
1,"""Apfel Boskoop ""Thurgauer""",Äpfel CH
2,"""Aprikosen ""ROUGECOT""",Aprikosen
3,"""Aprikosen ""VANILLACOT""",Aprikosen
4,"""Aprikosen AAA ""Walliser""",Aprikosen


In [41]:
df.reset_index(drop=True, inplace=True)

In [42]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Get the German stop words
stop_words = stopwords.words('german')

def clean_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    tokens = word_tokenize(text)
    # Removing stop words
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    # Removing numbers
    #tokens = [word for word in tokens if not word.isdigit()]
    # Removing extra whitespaces
    text = ' '.join(tokens)
    return text
df['DESCRIPTION']= df['DESCRIPTION'].apply(clean_text)
#df['Herstellung']=df['Herstellung'].apply(clean_text)
df['Herstellung']=df['Herstellung'].str.strip().str.lower()

In [43]:
df.head()

,DESCRIPTION,Herstellung
0,smoothfood brot 48x70g,"brot, übrige"
1,apfel boskoop thurgauer,äpfel ch
2,aprikosen rougecot,aprikosen
3,aprikosen vanillacot,aprikosen
4,aprikosen aaa wallis,aprikosen


In [44]:
df.Herstellung.value_counts()

Herstellung
nonfood                         1941
weizenbrot                      1097
hartkäse                        1083
milchspeiseeis                   906
saucen                           889
                                ... 
seehase, wild                      1
schweinefleisch, kat. 1, it        1
flunder, wild                      1
schweinefleisch, kat. 2, it        1
rindfleisch, saltimbocca, ch       1
Name: count, Length: 343, dtype: int64

In [46]:
from sklearn.preprocessing import LabelEncoder
# Encode the target column
label_encoder = LabelEncoder()
df["Herstellung"] = label_encoder.fit_transform(df["Herstellung"])
#df=df.drop("Herstellung_encoded", axis=1)
# Display the updated DataFrame
print(df.head())

               DESCRIPTION  Herstellung
0   smoothfood brot 48x70g           36
1  apfel boskoop thurgauer          341
2       aprikosen rougecot            6
3     aprikosen vanillacot            6
4     aprikosen aaa wallis            6


In [47]:
# Features & Labels
xlabels = df['DESCRIPTION']
ylabels = df['Herstellung']

In [24]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/bhauryal7/Multiclass-Text-Classification.mlflow')
dagshub.init(repo_owner='bhauryal7', repo_name='Multiclass-Text-Classification', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=da800e59-5f79-4fdf-a569-c3cd6c9cbb6c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=d222751c1726f070baed40b11d6db780e8848fdfb7e6dca0bc53584b13f17798




Accessing as bhauryal7

Initialized MLflow to track repo "bhauryal7/Multiclass-Text-Classification"

Repository bhauryal7/Multiclass-Text-Classification initialized!

2025/03/11 11:40:33 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/171f642d84fb4a49ac2029147c9fe969', creation_time=1741689633174, experiment_id='0', last_update_time=1741689633174, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [48]:
vectorizer = TfidfVectorizer(max_features=20000)
X= vectorizer.fit_transform(xlabels)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)


In [53]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "TfidfVectorizer")
        mlflow.log_param("num_features", 20000)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(C=10,max_iter=100,solver='liblinear',class_weight='balanced')  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)

        class_report = classification_report(y_test,y_pred)

        print("\nClassification Report:\n", class_report)

        # Log metrics in MLflow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_text(class_report, "classification_report.txt")

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-03-11 14:27:22,251 - INFO - Starting MLflow run...
2025-03-11 14:27:22,451 - INFO - Logging preprocessing parameters...
2025-03-11 14:27:23,040 - INFO - Initializing Logistic Regression model...
2025-03-11 14:27:23,052 - INFO - Fitting the model...
2025-03-11 14:27:52,446 - INFO - Model training complete.
2025-03-11 14:27:52,447 - INFO - Logging model parameters...
2025-03-11 14:27:52,664 - INFO - Making predictions...
2025-03-11 14:27:52,718 - INFO - Calculating evaluation metrics...
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels 


Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.80      0.76        10
           1       0.88      1.00      0.93        21
           2       0.92      0.60      0.73        20
           3       0.89      1.00      0.94         8
           4       0.75      1.00      0.86         3
           5       0.75      1.00      0.86         3
           6       1.00      0.91      0.95        22
           7       0.73      0.73      0.73        11
           8       0.71      0.71      0.71         7
           9       1.00      0.57      0.73         7
          10       1.00      1.00      1.00         1
          11       0.80      0.50      0.62         8
          12       0.25      1.00      0.40         1
          13       0.67      0.78      0.72        23
          14       0.68      0.77      0.72        22
          15       0.75      0.94      0.83        16
          16       0.82      0.87      0.84        31
  

2025-03-11 14:27:53,630 - INFO - Saving and logging the model...
2025/03/11 14:27:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-11 14:28:03,237 - INFO - Model training and logging completed in 40.79 seconds.


🏃 View run funny-penguin-69 at: https://dagshub.com/bhauryal7/Multiclass-Text-Classification.mlflow/#/experiments/0/runs/2ad4f62f59a24aa0bb0d2daf1932279b
🧪 View experiment at: https://dagshub.com/bhauryal7/Multiclass-Text-Classification.mlflow/#/experiments/0
